In [98]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [112]:
class UElasticNet:
    def __init__(self, alpha, l1_ratio, start_weights_type='optimal', gradient_type='default', step=None, 
                 epsilon=0.001, max_iter=1000, random_seed=101):
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.start_weights_type = start_weights_type
        self.gradient_type = gradient_type
        self.step = step
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.random_seed = random_seed

    def _startWeightsZeroes(self):
        return np.zeros(self.X.shape[1])

    def _startWeightsRandom(self):
        n = self.X.shape[1]
        return np.random.uniform(-1/(2*n), 1/(2*n), n)

    def _startWeightsOptimal(self):
        return (self.X.T @ self.y) / np.sum(self.X.T * self.X.T, axis=1)

    def _countGradientSG(self):
        next_i = np.random.randint(self.X.shape[0])
        x, y = self.X[next_i], self.y[next_i]
        vector_1 = x @ self.weights_ * x
        vector_2 = x * y
        return vector_1 - vector_2

    def _countGradient(self):
        vector_1 = self.X.T @ (self.X @ self.weights_)
        vector_2 = self.X.T @ self.y
        return (1 / self.X.shape[0]) * (vector_1 - vector_2)

    def fit(self, X, y):
        np.random.seed(self.random_seed)
        self.X = np.insert(X, 0, np.ones(X.shape[0]), axis=1)
        self.y = np.array(y)
        self.weights_ = self._startWeightsGens[self.start_weights_type](self)
        prev_weights = self.weights_ + np.ones(len(self.weights_))
        for k in range(1, self.max_iter+1):
            if np.linalg.norm(self.weights_-prev_weights) <= self.epsilon:
                break
            prev_weights = self.weights_
            h = (1 / np.sqrt(k)) if self.step is None else self.step
            self.weights_ = self.weights_ * (1 - h * self.alpha * (1 - self.l1_ratio)) \
                            - h * self._gradientType[self.gradient_type](self) \
                            - h * self.alpha * self.l1_ratio * np.sign(self.weights_)

    def predict(self, X):
        return X @ self.weights_[1:] + self.weights_[0]

    _startWeightsGens = {'zeroes': _startWeightsZeroes,
                         'random': _startWeightsRandom,
                         'optimal': _startWeightsOptimal}

    _gradientType = {'sg': _countGradientSG,
                     'default': _countGradient}

In [3]:
data = pd.read_csv('DATA/Advertising.csv')

In [4]:
X = data.drop('sales', axis=1)
y = data['sales']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [6]:
scaler = StandardScaler()

In [7]:
scaler.fit(X_train)

StandardScaler()

In [8]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = LinearRegression()

In [10]:
model.fit(X_train, y_train)

LinearRegression()

In [11]:
y_predict = model.predict(X_test)

In [12]:
np.sqrt(mean_squared_error(y_test, y_predict))

np.float64(1.5161519375993882)

In [13]:
model.intercept_, model.coef_

(np.float64(14.311428571428571),
 array([ 3.76599021,  2.76548662, -0.00690986]))

In [154]:
model = UElasticNet(alpha=0.7, l1_ratio=1, gradient_type='default', max_iter=10000, random_seed=42)

In [155]:
model.fit(X_train, y_train)

In [156]:
y_predict = model.predict(X_test)

In [157]:
np.sqrt(mean_squared_error(y_test, y_predict))

np.float64(1.9371493107231885)

In [149]:
model.weights_

array([1.40114286e+01, 3.48529289e+00, 2.48311874e+00, 2.67196071e-03])